In [18]:
import pandas as pd
import random

# Load your data
# Assuming the dataset is in a CSV file, replace this with your actual file path
data = pd.read_csv('poi_nearest_streets_99.csv')

# Step 1: Filter the data where 'is_on_street' is False
filtered_data = data[data['is_on_street'] == False]


# Step 3: Take the first 400 points from the filtered data
first_400_data = filtered_data.head(400)
f2 = first_400_data

f2.loc[:149, 'DISPLAY_LA'] = f2.loc[:149, 'ROUTING_LA']
f2.loc[:149, 'DISPLAY_LO'] = f2.loc[:149, 'ROUTING_LO']

# Step 3: Save the filtered data to another CSV file
geometry_index = f2.columns.get_loc('geometry')
f3 = f2.iloc[:, :geometry_index+1]
f3.to_csv('filtered_data_updated.csv', index=False)
print("Filtered data updated and saved as 'filtered_data_updated.csv'.")

# Step 4: Create the final CSV format with adjusted coordinates
def adjust_coordinates(lat, lon, max_offset=0.0001):
    # Adjust the coordinates slightly (0-6 meters)
    offset_lat = random.uniform(-max_offset, max_offset)
    offset_lon = random.uniform(-max_offset, max_offset)
    return lat + offset_lat, lon + offset_lon

# Create the new data (without OSM IDs)
final_data = []
for i, row in first_400_data.iterrows():
    house_number = row['HOUSE_NUMB'] if pd.notnull(row['HOUSE_NUMB']) else 'Unknown'
    street_name = row['STREET_NAM'] if pd.notnull(row['STREET_NAM']) else 'Unknown'
    lat, lon = adjust_coordinates(row['DISPLAY_LA'], row['DISPLAY_LO'])
    
    final_data.append([house_number, street_name, lon, lat])

# Convert the final data to a DataFrame
final_df = pd.DataFrame(final_data, columns=['house_number', 'street_name', 'longitude', 'latitude'])

# Step 5: Save the final dataframe to a CSV file
final_df.to_csv('final_output.csv', index=False)

print("CSV file with the required format has been saved as 'final_output.csv'.")




Filtered data updated and saved as 'filtered_data_updated.csv'.
CSV file with the required format has been saved as 'final_output.csv'.


/var/folders/y1/4lqd5nbn40s5kll78htdr9480000gn/T/ipykernel_13590/2482729433.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('poi_nearest_streets_99.csv')


In [22]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Function to convert CSV to GeoJSON
def csv_to_geojson(csv_file, geojson_file, lat_column, lon_column):
    # Load the CSV data
    data = pd.read_csv(csv_file)

    # Ensure the latitude and longitude columns exist
    if lat_column not in data.columns or lon_column not in data.columns:
        raise ValueError(f"Columns '{lat_column}' or '{lon_column}' not found in the CSV file.")

    # Create a GeoDataFrame with POINT geometries using latitude and longitude
    geometry = [Point(lon, lat) for lon, lat in zip(data[lon_column], data[lat_column])]
    geo_data = gpd.GeoDataFrame(data, geometry=geometry)

    # Save the GeoDataFrame to GeoJSON format
    geo_data.to_file(geojson_file, driver="GeoJSON")
    print(f"GeoJSON file saved as '{geojson_file}'.")


# # Convert filtered_data_updated.csv to GeoJSON
# csv_to_geojson('filtered_data_updated.csv', 'dummy.geojson', 'DISPLAY_LA', 'DISPLAY_LO')
# csv_to_geojson('cleaned_POIS_data.csv', 'cleaned.geojson', 'DISPLAY_LA', 'DISPLAY_LO')

# # Convert final_output.csv to GeoJSON
# csv_to_geojson('final_output.csv', 'OSMv2.geojson', 'latitude', 'longitude')

csv_to_geojson('output_weighted_coordinates.csv', 'output_weighted_coordinates.geojson', 'weighted_latitude', 'weighted_longitude')


GeoJSON file saved as 'output_weighted_coordinates.geojson'.


/Users/ompatil/VSC/Python/.venv/lib/python3.13/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
